# GOES-R Solar Irradiance
The GOES-R series have been monitoring atmospheric conditions and space weather since 1975. They are operated by the National Oceanic and Atmospheric Administration (NOAA) with the support of the National Aeronautic and Space Administration (NASA). 
https://www.goes-r.gov/

## About
[GOES-R Extreme Ultraviolet and X-ray Irradiance Sensors (EXIS)](https://www.ncei.noaa.gov/products/goes-r-extreme-ultraviolet-xray-irradiance)
>The Geostationary Operational Environmental Satellite (GOES)-R Extreme Ultraviolet and X-Ray Irradiance Sensors (EXIS) suite consists of two main instruments, the Extreme Ultraviolet Sensor (EUVS) and the X-Ray Sensor (XRS). EUVS products include high-cadence and time-averaged measurements of irradiance for seven extreme ultraviolet (EUV) solar lines, the magnesium II core-to-wing ratio (Mg II index), and modeled EUV spectra. 


# Extreme Ultraviolet Sensors (EUVS) Data

## Lyman-Alpha (EUV Proxy)
The **Lyman-Alpha (121.6nm)** is the brightest solar wavelength in the ultraviolet. It is emitted by the hydrogen in the Sun's atmosphere. It has been found to have corralation to the square root of the F10.7 index. 

### How Lyman-Alpha corralates to the F10.7 index

>"The second model (used when the Mg II index is not available) is based on the 10.7‐cm solar radio flux (F10.7) measurements made near Ottawa and Penticton (Canada) (Tapping, 2013). The Lyman α composite goes back to 1947 with the use of the F10.7 proxy... The F10.7 proxy is based on the square root of the F10.7, because the square root of the F10.7 has a better correlation with the Lyman α measurements."
>- Machol, J., Snow, M., Woodraska, D., Woods, T., Viereck, R., & Coddington, O. (2019). An improved lyman‐alphacomposite. Earth and Space Science, 6, 2263–2272. https://doi.org/10.1029/2019EA000648

## GOES-15
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-1-15/space-weather-instruments)
- **File:** G15_EUVE_daily_2010_2016_v4.txt
- **Product:**  1-min & Daily Averages (version 4)
     - EUVS channel E (121.6nm) 1-min and 1-day averaged counts, irradiances and flags
- **Date:** 2010-2016

**Note:**
we will need to  multiple the degration corrected average irradiance (irrad_ly) by AU_corr to correct the irradiance given that the distance between the Earth and Sun varies throughout the year. 

In [45]:
import pandas as pd


path = "datasets/G15_EUVE_daily_2010_2016_v4.txt"

df = pd.read_csv(
    path,
    comment=";",
    sep=r"\s+",
    header=None,
    names=["date", "Julday", "counts", "flags", "num", "irrad", "irrad_ly", "au_corr"],
    na_values=["-999.0", "-999"],
    dtype=str
)


# Change irrad_ly and au_corr to numeric
df[["irrad_ly", "au_corr"]] = df[["irrad_ly", "au_corr"]].apply(pd.to_numeric, errors="coerce")

# Create datetime column
df["date"] = pd.to_datetime(df["date"], errors="coerce")

#1 AU correction
df = pd.eval("irrad_corr = df.irrad_ly * df.au_corr", target=df)

# Drop bad/missing data
df = df.dropna().reset_index(drop=True)

# Drop uneccessary column
df = df.drop(["Julday", "counts", "flags", "num", "irrad",], axis=1)

#save as csv
df.to_csv("datasets/GOES15_average_irradiance.csv", index=False)

print(df)

           date  irrad_ly   au_corr  irrad_corr
0    2010-04-07  0.006309  1.000411    0.006312
1    2010-04-08  0.006492  1.000987    0.006498
2    2010-04-09  0.006475  1.001563    0.006485
3    2010-04-10  0.006446  1.002138    0.006460
4    2010-04-11  0.006424  1.002714    0.006441
...         ...       ...       ...         ...
2195 2016-06-02  0.006737  1.028364    0.006928
2196 2016-06-03  0.006713  1.028679    0.006906
2197 2016-06-04  0.006695  1.028986    0.006889
2198 2016-06-05  0.006666  1.029284    0.006861
2199 2016-06-06  0.006651  1.029572    0.006848

[2200 rows x 4 columns]


# GOES-16
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-r-extreme-ultraviolet-xray-irradiance)
- **File:** sci_euvs-l2-avg1d_g16_s20170207_e20250406_v1-0-6.nc
- **Product:**  EUVS Daily Average
     - Daily averages of spectral line irradiances, the Mg II index, and proxy spectra.
- **Date:** 2017-2025

In [9]:
import netCDF4 as nc
import xarray as xr
import pandas as pd


path = "datasets/sci_euvs-l2-avg1d_g16_s20170207_e20250406_v1-0-6.nc"

#convert to dataframe
ds = xr.open_dataset(path, engine="h5netcdf")
df = ds.to_dataframe()
print(df)



                                         irr_256   irr_284   irr_304  \
time       lines bounds wavelength_bin                                 
2017-02-07 0     0      0               0.000073  0.000020  0.000447   
                        1               0.000073  0.000020  0.000447   
                        2               0.000073  0.000020  0.000447   
                        3               0.000073  0.000020  0.000447   
                        4               0.000073  0.000020  0.000447   
...                                          ...       ...       ...   
2025-04-06 6     1      18              0.000166  0.000169  0.000667   
                        19              0.000166  0.000169  0.000667   
                        20              0.000166  0.000169  0.000667   
                        21              0.000166  0.000169  0.000667   
                        22              0.000166  0.000169  0.000667   

                                        irr_1175  irr_1216  irr

# X-Ray Sensors Data

## Solar Flares

## GOES-15
- **Source:** [NOAA](https://www.ncei.noaa.gov/products/goes-1-15/space-weather-instruments)
- **File:** sci_xrsf-l2-flsum_g15_s20100407_e20200304_v2-3-0.nc
- **Product:**  Flare Summary
     - List of solar flares with times, flare classes and integrated fluxes
- **Date:** 2010-2020
